In [1]:
import pandas as pd
from pyspark.sql import SparkSession, functions as F

In [2]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 1")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/08/01 10:32:34 WARN Utils: Your hostname, LAPTOP-RELH58H1 resolves to a loopback address: 127.0.1.1; using 172.19.22.4 instead (on interface eth0)
23/08/01 10:32:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/01 10:32:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Step 1: Provide the path to the CSV file
csv_file_path = "../data/raw/NYC_Vehicle_Collisions_Crashes.csv"

# Step 2: Read the CSV file into a Spark DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Step 3: Write the DataFrame to a Parquet file
parquet_output_path = "../data/raw/NYC_Vehicle_Collisions_Crashes.parquet"
df.write.parquet(parquet_output_path)


23/08/01 10:33:29 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


AnalysisException: [PATH_ALREADY_EXISTS] Path file:/mnt/c/Users/61435/Desktop/Applied Data/Project1/mast30034-project-1-HFISHER00/data/raw/NYC_Vehicle_Collisions_Crashes.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

REMOVING CRASHES WITH NULL FOR BOROUGH (LOCATION)

In [5]:
from pyspark.sql.functions import col

# Create a Spark session
spark = SparkSession.builder.appName("Remove Null Rows").getOrCreate()

# Read the parquet file into a DataFrame
Crash_results = spark.read.parquet('../data/raw/NYC_Vehicle_Collisions_Crashes.parquet')

# Filter out rows with null values in the "BOROUGH" column
Crash_results_filtered = Crash_results.na.drop(subset=["BOROUGH"])

# Show the filtered DataFrame
Crash_results_filtered.show()

# Save the filtered DataFrame to a new Parquet file
Crash_results_filtered.write.parquet('../data/landing/Crash_results_filtered.parquet', mode='overwrite')

+----------+----------+-------------+--------+---------+----------+--------------------+--------------------+------------------+--------------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+--------------------+--------------------+-------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME|      BOROUGH|ZIP CODE| LATITUDE| LONGITUDE|            LOCATION|      ON STREET NAME| CROSS STREET NAME|     OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRIBU